# Project 5: Deadpan Dog Analysis
An analysis, *sans* emotion, of the overemotional rating of dogs by the WeRateDogs twitter account.

*This project is for Unit 4, Data Wrangling, of the Data Analyst Nanodegree from Udacity*

## Table of Contents
<ul>
<li><a href='#intro'>Introduction</a></li>
<li><a href='#wrangling'>Data Wrangling</a></li>
    <ul>
    <li><a href='#gather'>Gathering Data</a></li>
    <li><a href='#assess'>Assessing Data</a></li>
    <li><a href='#clean'>Cleaning Data</a></li>
    </ul>
<li><a href='#eda'>Exploratory Data Analysis</a></li>
<li><a href='#stats'>Statistical Analysis</a></li>
<li><a href='#conclusions'>Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction
In this project, I will be working with the WeRateDogs Twitter account's tweets. This is a combination of the following sources:

- Archived Twitter Data (provided as a "file on hand" by Udacity)
- Image Predictions for Tweets (requested from a Udacity-hosted neural network)
- Additional Tweet Metadata (queried via Twitter API)

### Questions
1. What are Dogs?
2. How to doggo?
3. Why floof?

<a id='wrangling'></a>
## Data Wrangling
<ul>
<li><a href='#gather'>Gathering Data</a></li>
<li><a href='#assess'>Assessing Data</a></li>
<li><a href='#clean'>Cleaning Data</a></li>
</ul>

<a id='gather'></a>
### Gathering Data

<a id='assess'></a>
### Assessing Data

#### Issues

##### Quality
1. i
2. i
3. i
4. i
5. i
6. i
7. i
8. i

##### Tidiness
1. i
2. i

<a id='clean'></a>
### Cleaning Data

<a id='eda'></a>
## Exploratory Data Analysis

<a id='stats'></a>
## Statistical Analysis

<a id='conclusions'></a>
## Conclusions